# Limpeza dos dados

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Carregar os dados processados de 2010
df = pd.read_csv('../../dados/brutos/merged_2015.csv', low_memory=False)

In [ ]:
# Escolher as colunas para retirar
cols_to_drop = [
  # Colunas ENEM por escola
  'NU_ANO', # Ano da prova
  'CO_UF_ESCOLA', # Código do estado
  'CO_MUNICIPIO_ESCOLA', # Código do município
  'NO_MUNICIPIO_ESCOLA', # Nome do município
  'CO_ESCOLA_EDUCACENSO', # Código da escola
  'NO_ESCOLA_EDUCACENSO', # Nome da escola
  'TP_DEPENDENCIA_ADM_ESCOLA', # Dependência da escola / presente no censo
  'TP_LOCALIZACAO_ESCOLA', # Localização da escola / presente no censo
  'NU_MEDIA_OBJ', # Apenas em 2008
  'NU_MEDIA_TOT', # Apenas de 2005 a 2007

  # Colunas Censo 2015
  'NU_ANO_CENSO', # Ano do censo
  'NO_ENTIDADE', # Nome da escola
  'CO_ORGAO_REGIONAL', # Código do órgão regional de ensino
  'DT_ANO_LETIVO_INICIO', # Data de início do ano letivo / evitar análise temporal
  'DT_ANO_LETIVO_TERMINO', # Data de término do ano letivo
  'CO_REGIAO', # Código da região / foi deixado o estado
  'CO_MESORREGIAO', # Código da mesorregião / ^
  'CO_MICRORREGIAO', # Código da microrregião / ^
  'CO_UF', # Código do estado / foi deixado as siglas dos estados
  'CO_MUNICIPIO', # Código do município / evitar exagero de atributos
  'CO_DISTRITO', # Código do distrito
  'CO_ESCOLA_SEDE_VINCULADA', # Código da escola sede
  'CO_IES_OFERTANTE', # Código da IES vinculada
  'CO_LINGUA_INDIGENA', # Código da língua indígena que o ensino é ministrado
]

df.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
# % de presença de cada coluna no conjunto de dados
(1.-(df.isna().sum()[df.isna().sum() > 0] / df.shape[0])).sort_values() * 100

TP_INDIGENA_LINGUA                0.108988
TP_OCUPACAO_GALPAO                0.230799
TP_CONVENIO_PODER_PUBLICO         3.833825
IN_MANT_ESCOLA_PRIVADA_S_FINS    40.550071
IN_MANT_ESCOLA_PRIVADA_SIST_S    40.550071
IN_MANT_ESCOLA_PRIVADA_SIND      40.550071
IN_MANT_ESCOLA_PRIVADA_EMP       40.550071
IN_MANT_ESCOLA_PRIVADA_ONG       40.550071
TP_CATEGORIA_ESCOLA_PRIVADA      40.550071
IN_CONVENIADA_PP                 40.550071
IN_FUNDAMENTAL_CICLOS            74.791640
TP_OCUPACAO_PREDIO_ESCOLAR       99.301192
IN_PREDIO_COMPARTILHADO          99.346070
NU_TAXA_ABANDONO                 99.474292
NU_TAXA_REPROVACAO               99.474292
NU_TAXA_APROVACAO                99.474292
NU_SALAS_EXISTENTES              99.846134
PC_FORMACAO_DOCENTE              99.878190
INSE                             99.993589
dtype: float64

In [ ]:
# Remover colunas majoritariamente nulas
too_much_na = [
  'TP_INDIGENA_LINGUA', # Língua de ensino
  'TP_OCUPACAO_GALPAO', # Forma de ocupação do galpão, se aplicável
  'TP_CONVENIO_PODER_PUBLICO', # Tipo de convênio com o poder público, se aplicável
  # Colunas abaixo se referem apenas a escolas privadas
  # (a decisão foi de removê-las)
  'IN_MANT_ESCOLA_PRIVADA_S_FINS',
  'IN_MANT_ESCOLA_PRIVADA_SIST_S',
  'IN_MANT_ESCOLA_PRIVADA_SIND',
  'IN_MANT_ESCOLA_PRIVADA_EMP',
  'IN_MANT_ESCOLA_PRIVADA_ONG',
  'TP_CATEGORIA_ESCOLA_PRIVADA',
  'IN_CONVENIADA_PP',
]

df.drop(too_much_na, axis=1, inplace=True)

In [ ]:
# % de presença de cada coluna no conjunto de dados
(1.-(df.isna().sum()[df.isna().sum() > 0] / df.shape[0])).sort_values() * 100

IN_FUNDAMENTAL_CICLOS         74.791640
TP_OCUPACAO_PREDIO_ESCOLAR    99.301192
IN_PREDIO_COMPARTILHADO       99.346070
NU_TAXA_APROVACAO             99.474292
NU_TAXA_REPROVACAO            99.474292
NU_TAXA_ABANDONO              99.474292
NU_SALAS_EXISTENTES           99.846134
PC_FORMACAO_DOCENTE           99.878190
INSE                          99.993589
dtype: float64

In [ ]:
# Realizar o one-hot encoding nas colunas consideradas categóricas
cat_cols = [
  # ENEM por escola
  'SG_UF_ESCOLA',
  'INSE', # (Grupo 1 - Grupo 6)
  'PORTE_ESCOLA', # Porte de tamanho da escola

  # Censo 2015
  'TP_SITUACAO_FUNCIONAMENTO', # Situação de funcionamento (1 - 4)
  'TP_DEPENDENCIA', # Dependência administrativa (1 - 4)
  'TP_REGULAMENTACAO', # Estado da regulamentação (0 - 2)
  'TP_OCUPACAO_PREDIO_ESCOLAR', # Forma de ocupação do prédio escolar (1 - 3)
  'TP_AEE', # Atendimento Educacional Especializado (0 - 2)
  'TP_ATIVIDADE_COMPLEMENTAR', # Atividade complementar (0 - 2)
  'TP_LOCALIZACAO_DIFERENCIADA', # Localização diferenciada da escola (0 - 6)
]

# OBS: dummies acaba removendo as observações com valores nulos nas colunas aplicadas
df = pd.get_dummies(df, prefix=cat_cols, columns=cat_cols)

In [ ]:
# Mudança do encoding da localização de escola de 1-2 para 0-1
df.loc[(df['TP_LOCALIZACAO'] == 1), 'TP_LOCALIZACAO'] = 0 # Urbana
df.loc[(df['TP_LOCALIZACAO'] == 2), 'TP_LOCALIZACAO'] = 1 # Rural

In [ ]:
# Adicionar indicador de que houve valor faltante preenchido
df.loc[~df.isna().any(axis=1), 'VALOR_FALTANTE'] = 0
df.loc[df.isna().any(axis=1), 'VALOR_FALTANTE'] = 1

In [ ]:
# Reorganizar para que as colunas das médias fiquem no começo
mean_cols = ['NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_MT', 'NU_MEDIA_RED']

df = df.reindex(mean_cols + [*df.drop(mean_cols, axis=1).columns], axis=1)

In [ ]:
# Imputação dos dados faltantes utilizando KNN
from sklearn.impute import KNNImputer

imputer = KNNImputer()
df_transformed = imputer.fit_transform(df)

In [ ]:
# Reconstrução do conjunto
df_imputed = pd.DataFrame(df_transformed, columns=df.columns)

In [ ]:
# Remover as colunas com variância nula (mesmo valor na coluna inteira)
df_clean = df_imputed.loc[:,~(df_imputed.var() == 0)]

In [ ]:
# Exportar o .csv finalizado
df_clean.to_csv('../../dados/processados/2015_clean.csv', index=False)